**Специальные методы** (*special methods*), также называемые **магическими методами** или **dunder-методами** (от *double underscore*) – это методы с именами вида `__name__()`, которые Python вызывает неявно при выполнении определённых операций над объектом. Например, выражение `a + b` транслируется в вызов `a.__add__(b)`, а функция `len(obj)` – в вызов `obj.__len__()`. Определяя эти методы в своём классе, программист задаёт, как объекты этого класса будут взаимодействовать со встроенными операторами (`+`, `<`, `in`, ...), функциями (`len()`, `str()`, `repr()`, ...) и синтаксическими конструкциями (индексация, итерация, контекстные менеджеры и др.). Полный список специальных методов описан в разделе [Data Model](https://docs.python.org/3/reference/datamodel.html#special-method-names) документации Python.


Неплохой гайд по магическим методам представлен в этой [статье](https://habr.com/ru/articles/186608/).

Список магических методов представлен [здесь](https://mathspp.com/blog/pydonts/dunder-methods).

## Методы перехвата доступа к атрибутам объекта

Следующие методы вызываются при взаимодействии с атрибутами объектов:

- `__setattr__(self, name, value)` – при создании или изменении атрибута;
- `__getattribute__(self, name)` – при **любом** обращении к атрибуту;
- `__getattr__(self, name)` – вызывается, только если `__getattribute__()` не нашёл атрибут (т.е. возбудил `AttributeError`);
- `__delattr__(self, name)` – при удалении атрибута.

При переопределении методов `__setattr__()`, `__getattribute__()` и `__delattr__()` необходимо вызывать одноимённый метод суперкласса, чтобы сохранить стандартное поведение. Обычно используют `super().__setattr__(name, value)` или напрямую `object.__setattr__(self, name, value)`.

> ⚠️ Присваивание атрибута внутри метода `__setattr__()` через `self.name = value` приведёт к бесконечной рекурсии, поскольку каждое присваивание вызывает `__setattr__()` снова. Чтобы этого избежать, используйте `object.__setattr__(self, name, value)`.

По умолчанию Python позволяет динамически добавлять новые атрибуты к объектам. Чтобы ограничить набор допустимых атрибутов, можно определить атрибут класса `allowed_attrs`, содержащий имена разрешённых атрибутов, и в методе `__setattr__()` возбуждать исключение `AttributeError` при попытке присвоить значение атрибуту, не входящему в этот список:

In [5]:
class Cell:
   
    # Разрешённые атрибуты
    allowed_attrs = {'size', 'membrane_type', 'is_alive', 'energy'}
    
    def __init__(self, size, membrane_type="липидный бислой"):
        self.size = size
        self.membrane_type = membrane_type
        self.is_alive = True
        self.energy = 100
    
    def __getattribute__(self, name):
        """Вызывается при ЛЮБОМ обращении к атрибуту"""
        print(f"  [__getattribute__] Запрос атрибута '{name}'")
        # Обязательно вызываем метод родителя, иначе ничего не получим
        return object.__getattribute__(self, name)
    
    def __getattr__(self, name):
        """Вызывается ТОЛЬКО если атрибут не найден"""
        print(f"  [__getattr__] Атрибут '{name}' не существует!")
        # Возвращаем значение по умолчанию или возбуждаем исключение
        if name == 'age':
            return 0  # значение по умолчанию для возраста
        raise AttributeError(f"У клетки нет атрибута '{name}'")
    
    def __setattr__(self, name, value):
        """Вызывается при создании или изменении атрибута"""
        print(f"  [__setattr__] Установка '{name}' = {value}")
        
        # Проверяем, разрешён ли атрибут
        # Используем object.__getattribute__, чтобы избежать вызова нашего __getattribute__
        allowed = object.__getattribute__(self, '__class__').allowed_attrs
        
        if name not in allowed:
            raise AttributeError(f"Нельзя создать атрибут '{name}'. "
                                 f"Разрешены только: {allowed}")
        
        # Используем object.__setattr__, чтобы избежать рекурсии
        object.__setattr__(self, name, value)

In [10]:
cell = Cell(10)

  [__setattr__] Установка 'size' = 10
  [__setattr__] Установка 'membrane_type' = липидный бислой
  [__setattr__] Установка 'is_alive' = True
  [__setattr__] Установка 'energy' = 100


In [11]:
print(f"Размер клетки: {cell.size}")    # чтение существующего атрибута

  [__getattribute__] Запрос атрибута 'size'
Размер клетки: 10


In [14]:
# чтение НЕсуществующего атрибута (с значением по умолчанию) 
print(f"Возраст клетки: {cell.age}")    

  [__getattribute__] Запрос атрибута 'age'
  [__getattr__] Атрибут 'age' не существует!
Возраст клетки: 0


In [19]:
# чтение НЕсуществующего атрибута (без значения по умолчанию) 
print(f"Цвет клетки: {cell.color}")

  [__getattribute__] Запрос атрибута 'color'
  [__getattr__] Атрибут 'color' не существует!


AttributeError: У клетки нет атрибута 'color'

In [20]:
cell.energy = 50    # изменение существующего атрибута

  [__setattr__] Установка 'energy' = 50


Метод `__setattr__()` вызывается и при присвоении значения несуществующему атрибуту. По-умолчанию, у объекта создается новый атрибут. В методе `__setattr__()` класса `Cell` мы изменили это поведение: новый атрибут будет создан только в том случае, если его имя содержится в поле класса `allowed_attrs`, который содержит разрешенные имена:

In [23]:
cell.age = 10

  [__setattr__] Установка 'age' = 10


AttributeError: Нельзя создать атрибут 'age'. Разрешены только: {'energy', 'membrane_type', 'size', 'is_alive'}

## Жизненный цикл объекта

При вызове `Cell()` Python выполняет два действия:

1. **Создание** (*construction*): вызывается `__new__()`, который выделяет память и возвращает пустой экземпляр;
2. **Инициализация** (*initialization*): вызывается `__init__()`, который заполняет атрибуты созданного экземпляра.

При удалении объекта (когда на него больше нет ссылок) вызывается `__del__()`.


### Метод `__new__()`

Метод `__new__(cls, *args, **kwargs)` отвечает за создание нового экземпляра класса. Формально это статический метод (*static method*), однако Python автоматически передаёт ему класс первым аргументом (`cls`), за которым следуют те же аргументы, что передаются в `__init__()`. Это делает поведение `__new__()` похожим на метод класса. 

Метод должен возвращать экземпляр класса. Обычно для этого вызывают `super().__new__(cls)` (для наследников `object` дополнительные аргументы передавать не нужно). Возвращаемый экземпляр ещё не инициализирован – его `__dict__` пуст.

При создании неизменяемых типов данных, их значения определяются при создании объекта, а не при инициализации.

Переопределение `__new__()` требуется редко:
- при наследовании от неизменяемых типов – `int`, `str`, `tuple`, – поскольку их значение нужно задать до инициализации;
- при реализации паттерна *Singleton*;
- при создании метаклассов.

В качестве примера реализуем класс, который является **одиночкой** (*singleton*): у него может существовать только один экземпляр в приложении . Для этого определим поле класса `_instance` которое инициализируется значением `None`. При создании первого экземпляра класса в методе `__new__()` полю класса `_instance` будет присвоен сам объект (возвращаемый методом `__new__()` суперкласса). При последующих попытках создания нового объекта класса, будет возвращаться созданный уже экземпляр класса.

In [ ]:
class Singleton:
    _instance = None            # Атрибут класса, в котором будет храниться экземпляр

    def __new__(cls):
        if cls._instance is None:       # Проверяется, существует ли уже экземпляр
            # Создается объект и присваивается атрибуту класса:
            cls._instance = super().__new__(cls)
        return cls._instance

s1 = Singleton()
s2 = Singleton()
s1 is s2

### Метод `__del__(self)`

`__del__()` – **финализатор** (*finalizer*) вызывается, когда **счётчик ссылок** (*reference count*) на объект достигает нуля и объект удаляется сборщиком мусора.

In [27]:
class Cell:
    def __del__(self):
        print(f"объект {self} удаляется!")

L = Cell()
L = 12

объект <__main__.Cell object at 0x000001BFAD7429F0> удаляется!


⚠️ Оператор `del obj` **не гарантирует** немедленный вызов `__del__()`, а лишь уменьшает счётчик ссылок. Объект будет удалён только когда ссылок не останется. 

In [29]:
L = Cell()
M = L           # создается вторая ссылка на объект
del L           # удаляется ссылка L, но на объект 
M               # продолжает ссылаться M


- При наличии **циклических ссылок** (*circular references*) вызов `__del__()` может быть отложен или не произойти вовсе.
- Финализатор может **не быть вызван** при аварийном завершении интерпретатора.
- Исключения, возникшие в `__del__()`, игнорируются (выводится предупреждение в `stderr`).

> ⚠️ Не следует полагаться на `__del__()` для освобождения критических ресурсов (файлы, сетевые соединения, блокировки). Для этого лучше использовать контекстные менеджеры.

Метод `__del__()` не является обязательным. Встроенные типы (`list`, `dict`, `int`, ...) его не определяют, потому что им не нужна специальная логика при удалении – стандартного освобождения памяти достаточно.

## Поведение объекта

Модель данных Python позволяет пользовательским типам поддерживать те же операции, что и встроенные типы. Для этого достаточно реализовать соответствующие специальные методы. Например:

- взаимодействие со встроенными функциями (`len()`, `abs()`, `hash()`, `bool()` и др.);
- арифметические операторы (`+`, `-`, `*`, `/`, ...);
- операторы сравнения (`<`, `>`, `==`, ...);
- оператор принадлежности (`in`);
- доступ к элементам по индексу или ключу (`obj[key]`), как в списках и словарях;
- итерирование в цикле `for`;
- преобразование типов (`int()`, `float()`, `str()`, ...).

Набор специальных методов, реализующих определённое поведение, называется **протоколом** (*protocol*). Например, чтобы объект поддерживал итерирование, достаточно реализовать протокол итератора (Iterator) – методы `__iter__()` и `__next__()`, а для того, чтобы  реализовать протокол изменяемой последовательности (MutableSequence) – методы `__getitem__()`, `__setitem__()`, `__delitem__()` и `__len__()`.

## Строковое представление объекта

Специальные методы `__repr__()` и `__str__()` определяют, как объект преобразуется в строку. Хотя оба метода возвращают строковое представление объекта, они предназначены для разных целей.

In [32]:
class Cell:

    def __init__(self, size, cell_type="generic"):
        self.size = size
        self.cell_type = cell_type
        self.is_alive = True
    
    def __repr__(self):
        return f"Cell(size={self.size}, cell_type='{self.cell_type}')"
    
    def __str__(self):
        status = "alive" if self.is_alive else "dead"
        return f"{self.cell_type.capitalize()} cell ({self.size} μm, {status})"
    
cell = Cell(24)

### `__repr__()`

`__repr__(self)` возвращает формальное (*official*) строковое **представление** (*representation*) объекта, предназначенное для разработчика. В идеале это должна быть строка, по которой можно воссоздать объект (валидное Python-выражение), либо информативное описание в угловых скобках.

- Вызывается функцией `repr(obj)`.
- Используется в интерактивной консоли при выводе значения.
- Используется в отладчике.
- Применяется для элементов внутри контейнеров (`list`, `dict`, ...).

Если последняя строка в ячейке Jupyter представляет собой выражение, то после выполнения этого выражения на печать будет выведено формальное представление объекта.

In [34]:
repr(cell)

"Cell(size=24, cell_type='generic')"

In [33]:
cell

Cell(size=24, cell_type='generic')

### `__str__()`

`__str__(self)` возвращает неформальное (*informal*) строковое представление, предназначенное для пользователя. Должно быть читаемым и понятным.

- Вызывается функцией `str(obj)`.
- Используется функцией `print()`.
- Используется при форматировании строк (f-строки, `.format()`).

In [35]:
str(cell)

'Generic cell (24 μm, alive)'

In [36]:
print(cell)

Generic cell (24 μm, alive)


In [37]:
f"Клетка {cell}"

'Клетка Generic cell (24 μm, alive)'